In [0]:
from pyspark.sql.functions import col, from_json, regexp_extract, trim
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

# 1. Aggressive Schema (using String for Year first to avoid type-mismatch nulls)
schema = StructType([
    StructField("company_name", StringType(), True),
    StructField("founding_year", StringType(), True), # Load as string first!
    StructField("top_products", ArrayType(StringType()), True)
])

# 2. Extract ONLY the stuff between the first '{' and the last '}'
# This ignores all the AI's "Here is your JSON" conversational text.
json_regex = r"(\{.*\})" 

df_final = spark.table("silver.startup_fundamentals") \
    .withColumn("just_json", regexp_extract(col("structured_data"), json_regex, 1)) \
    .withColumn("parsed", from_json(col("just_json"), schema)) \
    .select(
        col("parsed.company_name").alias("Company"),
        col("parsed.founding_year").cast("int").alias("Founded"), # Cast to int here
        col("parsed.top_products").alias("Products")
    )

# 3. Final Check: If it's still null, let's see exactly what 'just_json' looks like
display(df_final)

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Calculate how many top models each startup has
SELECT 
  Company, 
  size(Models) AS Model_Count, 
  Founded
FROM gold_startup_analytics
ORDER BY Model_Count DESC